## Intro

Rust takes a lot of inspiration from functional languages. <span style="color:skyblue">*Programming in a functional style often includes using functions as values by passing them in arguments, returning them from other functions, assigning them to variables for later execution, and so forth.*</span>

In this chapter, we will cover:

- **Closures**, a function-like construct you can store in a variable
- **Iterators**, a way of processing a series of elements
- How to use closures and iterators to improve the I/O project in Chapter 12
- The performance of closures and iterators (Spoiler alert: they're faster than you might think!)


## Closures: Anonymous Functions that Capture Their Environment

<span style="color:skyblue">***Rust’s closures are anonymous functions you can save in a variable or pass as arguments to other functions. You can create the closure in one place and then call the closure elsewhere to evaluate it in a different context. Unlike functions, closures can capture values from the scope in which they’re defined. We’ll demonstrate how these closure features allow for code reuse and behavior customization.***</span>

### Capturing the Environment with Closures
*Scenario*: Every so often, our t-shirt company gives away an exclusive, limited-edition shirt to someone on our mailing list as a promotion. People on the mailing list can optionally add their favorite color to their profile. If the person chosen for a free shirt has their favorite color set, they get that color shirt. If the person hasn’t specified a favorite color, they get whatever color the company currently has the most of.

In [2]:
#[derive(Debug, PartialEq, Copy, Clone)]
enum ShirtColor {
    Red,
    Blue,
}

/// representing the shirt colors currently in stock. 
struct Inventory {
    shirts: Vec<ShirtColor>,
}

impl Inventory {
    /// gets the optional shirt color preference of the free shirt winner, 
    /// and returns the shirt color the person will get
    fn giveaway(&self, user_preference: Option<ShirtColor>) -> ShirtColor {
        user_preference.unwrap_or_else(|| self.most_stocked())
        // here, we used closure. We got the user preference as a 
        // parameter of type `Option<ShirtColor>` and call the 
        // `unwrap_or_else` method on `user_preference`. The
        // `unwrap_or_else` takes one argument: a closure without
        // any arguments, calls `self.most_stocked()` in the body 
        // that returns a value `T` of the calling `Option<T>`
        // (In this case, `T` is `ShirtColor`)
        // If the `Option<T>` is the `Some` variant, `unwrap_or_else` 
        // returns the value from within the `Some`. If the `Option<T>` 
        // is the `None` variant, `unwrap_or_else` calls the closure 
        // and returns the value returned by the closure.
        // Note: Here, the closure captures an immutable reference 
        // to the self `Inventory` instance and passes it with the 
        // code we specify to the `unwrap_or_else` method. 
        // Functions, on the other hand, are not able to capture 
        // their environment in this way.
    }

    /// find out if the store has more blue / red shirts in the stock
    fn most_stocked(&self) -> ShirtColor {
        let mut num_red = 0;
        let mut num_blue = 0;

        for color in &self.shirts {
            match color {
                ShirtColor::Red => num_red += 1,
                ShirtColor::Blue => num_blue += 1,
            }
        }

        if num_red > num_blue {
            ShirtColor::Red
        } else {
            ShirtColor::Blue
        }
    }
}

fn main() {
    let store = Inventory {
        shirts: vec![ShirtColor::Blue, ShirtColor::Red, ShirtColor::Blue],
    };

    let user_pref1 = Some(ShirtColor::Red);
    let giveaway1 = store.giveaway(user_pref1);
    println!(
        "The user with preference {:?} gets {:?}",
        user_pref1, giveaway1
    );

    let user_pref2 = None;
    let giveaway2 = store.giveaway(user_pref2);
    println!(
        "The user with preference {:?} gets {:?}",
        user_pref2, giveaway2
    );
}

main()

The user with preference Some(Red) gets Red
The user with preference None gets Blue


()

### Closure Type Inference and Annotation

Unlike functions, closures don’t usually require you to annotate the types of the parameters or the return value like `fn` functions do. This is because functions are often exposed to users so we need everyone to agree on the types of values; closures, on other other hand, are used without naming and are not exposed to users.

As with variables, we can add type annotations if we want to increase explicitness and clarity at the cost of being more verbose than is strictly necessary.

In [4]:
use std::thread;
use std::time::Duration;

fn generate_workout(intensity: u32, random_number: u32) {
    let expensive_closure = |num: u32| -> u32 {  // type annotation with
                                                 // closure here
        println!("calculating slowly...");
        thread::sleep(Duration::from_secs(2));
        num
    };

    if intensity < 25 {
        println!("Today, do {} pushups!", expensive_closure(intensity));
        println!("Next, do {} situps!", expensive_closure(intensity));
    } else {
        if random_number == 3 {
            println!("Take a break today! Remember to stay hydrated!");
        } else {
            println!(
                "Today, run for {} minutes!",
                expensive_closure(intensity)
            );
        }
    }
}

fn main() {
    let simulated_user_specified_value = 10;
    let simulated_random_number = 7;

    generate_workout(simulated_user_specified_value, simulated_random_number);
}

main()

calculating slowly...
Today, do 10 pushups!
calculating slowly...
Next, do 10 situps!


()

How closure syntax is similar to function syntax except for the use of pipes and the amount of syntax that is optional:

```rust
fn  add_one_v1   (x: u32) -> u32 { x + 1 }
let add_one_v2 = |x: u32| -> u32 { x + 1 };
let add_one_v3 = |x|             { x + 1 };  //  require the closures to be evaluated to be able to compile
let add_one_v4 = |x|               x + 1  ; //  require the closures to be evaluated to be able to compile
```

In [6]:
fn main() {
    let example_closure = |x| x;

    let s = example_closure(String::from("hello"));
    println!("s = {s}");
    // let n = example_closure(5);
}

main()

s = hello


()

### Capturing References or Moving Ownership

<span style="color:skyblue">*Closures can capture values from their environment in three ways: borrowing immutably, borrowing mutably, and taking ownership, which directly map to the three ways a function can take a parameter. The closure will decide which of these to use based on what the body of the function does with the captured values.*</span>

Below is an example of borrowing immutably:

In [8]:
fn main() {
    let list = vec![1, 2, 3];
    println!("Before defining closure: {:?}", list);

    let only_borrows = || println!("From closure: {:?}", list);

    println!("Before calling closure: {:?}", list);
    only_borrows();  // calling the closure
    println!("After calling closure: {:?}", list);
}

main()

Before defining closure: [1, 2, 3]
Before calling closure: [1, 2, 3]
From closure: [1, 2, 3]
After calling closure: [1, 2, 3]


()

Because we can have multiple immutable references to list at the same time, list is still accessible from the code before the closure definition, after the closure definition but before the closure is called, and after the closure is called.

Below, we make a closure that mutate its input:

In [15]:
fn main() {
    let mut list = vec![1, 2, 3];
    println!("Before defining closure: {:?}", list);

    let mut borrows_mutably = || list.push(7);

    // will throw error if print since immutable borrow 
    // to print isn’t allowed because no other borrows 
    // are allowed when there’s a mutable borrow.
    // println!("Before defining closure: {:?}", list);

    borrows_mutably();
    println!("After calling closure: {:?}", list);
}

main()

Before defining closure: [1, 2, 3]
After calling closure: [1, 2, 3, 7]


()

Use the `move` keyword to force the closure to take ownership of the values it uses in the environment even though the body of the closure doesn’t strictly need ownership

In [20]:
use std::thread;

fn main() {
    let list = vec![1, 2, 3];
    println!("Before defining closure: {:?}", list);

    thread::spawn(move || println!("From thread: {:?}", list)).join().unwrap();
}

main()

Before defining closure: [1, 2, 3]
From thread: [1, 2, 3]


()

We spawn a new thread, giving the thread a closure to run as an argument. The closure body prints out the list. The closure only captured `list` using an immutable reference because that's the least amount of access to `list` needed to print it. In this example, even though the closure body still only needs an immutable reference, we need to specify that `list` should be moved into the closure by putting the `move` keyword at the beginning of the closure definition. The new thread might finish before the rest of the main thread finishes, or the main thread might finish first. If the main thread maintained ownership of `list` but ended before the new thread did and dropped list, the immutable reference in the thread would be invalid. Therefore, the compiler requires that list be moved into the closure given to the new thread so the reference will be valid. Try removing the `move` keyword or using `list` in the main thread after the closure is defined to see what compiler errors you get!

In [25]:
use std::thread;

fn main() {
    let list = vec![1, 2, 3];
    println!("Before defining closure: {:?}", list);

    thread::spawn(|| println!("From thread: {:?}", list)).join().unwrap();
}

main()

Error: closure may outlive the current function, but it borrows `list`, which is owned by the current function

### Moving Captured Values Out of Closures and the `Fn` Traits

Once a closure has captured a reference or captured ownership of a value from the environment where the closure is defined, it can do the following:
- <span style="color:skyblue">*Move a captured value out of the closure*</span>
- <span style="color:skyblue">*Mutate the captured value*</span>
- <span style="color:skyblue">*Neither move nor mutate the value*</span>
- <span style="color:skyblue">*Or capture nothing from the environment to begin with*</span>

The way a closure captures and handles values from the environment affects which traits the closure implements, and traits are how functions and structs can specify what kinds of closures they can use. Closures will automatically implement one, two, or all three of these `Fn` traits, in an additive fashion, depending on how the closure’s body handles the values:

- <span style="color:skyblue">*`FnOnce` applies to closures that can be called once*</span>. All closures implement at least this trait, because all closures can be called. <span style="color:skyblue">*A closure that moves captured values out of its body will only implement `FnOnce` and none of the other `Fn` traits, because it can only be called once.*</span>
- <span style="color:skyblue">*`FnMut` applies to closures that don’t move captured values out of their body, but that might mutate the captured values. These closures can be called more than once.*</span>
- <span style="color:skyblue">*`Fn` applies to closures that don’t move captured values out of their body and that don’t mutate captured values, as well as closures that capture nothing from their environment.*</span> These closures can be called more than once without mutating their environment, which is important in cases such as <span style="color:skyblue">*calling a closure multiple times concurrently*</span>.


**Example using `FnOnce()`**: Below, we have the definition of the `unwrap_or_else` method on `Option<T>`. The trait bound specified on the generic type `F` is `FnOnce() -> T`, which means `F` must be able to be called once, take no arguments, and return a `T`. Using `FnOnce` in the trait bound expresses the constraint that `unwrap_or_else` is only going to call `f` at most one time.

```rust
impl<T> Option<T> {
    pub fn unwrap_or_else<F>(self, f: F) -> T
    where 
        F: FnOnce() -> T 
    {
        match self {
            Some(x) => x,
            None => f(), // f: F will only be called only once when Option is None
        }
    }
}
```

**Example using `FnMut`**: The std method `sort_by_key` defined on slices:

In [27]:
#[derive(Debug)]
struct Rectangle {
    width: u32,
    height: u32,
}

fn main() {
    let mut list = [
        Rectangle {width: 10, height: 1},
        Rectangle {width: 3, height: 5},
        Rectangle {width: 7, height: 12},
    ];

    list.sort_by_key(|r| r.width);
    println!("{:#?}", list);
}

main()

[
    Rectangle {
        width: 3,
        height: 5,
    },
    Rectangle {
        width: 7,
        height: 12,
    },
    Rectangle {
        width: 10,
        height: 1,
    },
]


()

In [29]:
#[derive(Debug)]
struct Rectangle {
    width: u32,
    height: u32,
}

fn main() {
    let mut list = [
        Rectangle { width: 10, height: 1 },
        Rectangle { width: 3, height: 5 },
        Rectangle { width: 7, height: 12 },
    ];

    let mut num_sort_operations = 0;
    list.sort_by_key(|r| {
        num_sort_operations += 1;
        r.width
    });
    println!("{:#?}, sorted in {num_sort_operations} operations", list);
}

main()

[
    Rectangle {
        width: 3,
        height: 5,
    },
    Rectangle {
        width: 7,
        height: 12,
    },
    Rectangle {
        width: 10,
        height: 1,
    },
], sorted in 6 operations


()

## Iterators: Processing a Series of Items

<span style="color:skyblue">*An iterator is responsible for the logic of iterating over each item and determining when a sequence has finished. When you use iterators, you don’t have to reimplement that logic yourself. In Rust, iterators are lazy, meaning they have no effect until you call methods that consume the iterator to use it up.*</span>

In [30]:
let v1 = vec![1, 2, 3];

let v1_iter = v1.iter();  // lazy and not doing anything here yet

for val in v1_iter {
    println!("Got: {}", val);
}

Got: 1
Got: 2
Got: 3


()

### The `Iterator` Trait and the `next` Method

All iterators implement a trait named `Iterator` that is defined in the standard library. 

```rust
pub trait Iterator {
    type Item;  // defining an associated type with this trait
                // needed for the return type of `next`

    fn next(&mut self) -> Option<Self::Item>;

    // methods with default implementations elided
}
```

We can call the `next` method on iterators directly

In [33]:
fn iterator_demonstration() {
    let v1 = vec![1, 2, 3];

    let mut v1_iter = v1.iter();

    assert_eq!(v1_iter.next(), Some(&1));
    assert_eq!(v1_iter.next(), Some(&2));
    assert_eq!(v1_iter.next(), Some(&3));
    assert_eq!(v1_iter.next(), None);
}
iterator_demonstration()

()

**Note:** <span style="color:green">*We needed to make `v1_iter` mutable: calling the `next` method on an iterator changes internal state that the iterator uses to keep track of where it is in the sequence. Also note that the values we get from the calls to `next` are immutable references to the values in the vector. If we want to create an iterator that takes ownership of `v1` and returns owned values, we can call `into_iter` instead of `iter`. Similarly, if we want to iterate over mutable references, we can call `iter_mut` instead of `iter`.*</span>

### Methods that Consume the Iterator

Methods that call next are called <span style="color:skyblue">***consuming adaptors***</span>, because calling them uses up the iterator. One example is the `sum` method, which takes ownership of the iterator and iterates through the items by repeatedly calling next, thus consuming the iterator.

In [38]:
fn iterator_sum() {
    let v1 = vec![1, 2, 3];

    let v1_iter = v1.iter();

    let total: i32 = v1_iter.sum();  // consuming adaptor `sum`

    println!("total = {total}");
    assert_eq!(total, 6);
}

iterator_sum()

total = 6


()

### Methods that Produce Other Iterators

<span style="color:skyblue">***Iterator adaptors***</span> are methods defined on the Iterator trait that don’t consume the iterator. Instead, they produce different iterators by changing some aspect of the original iterator.

In [42]:
let v1: Vec<i32> = vec![1, 2, 3];
println!("v1 = {:?}", v1);

let v2: Vec<_> = v1.iter().map(|x| x + 1).collect();
println!("v2 = {:?}", v2);


v1 = [1, 2, 3]
v2 = [2, 3, 4]


### Using Closures that Capture Their Environment

Many iterator adapters take closures as arguments, and commonly the closures we’ll specify as arguments to iterator adapters will be closures that capture their environment.

Example: Below, we use `filter` with a closure that captures the `shoe_size` variable from its environment to iterate over a collection of `Shoe` struct instances. It will return only shoes that are the specified size.

In [49]:
#[derive(PartialEq, Debug)]
struct Shoe {
    size: u32,
    style: String,
}

fn shoes_in_size(shoes: Vec<Shoe>, shoe_size: u32) -> Vec<Shoe> {
    // this function takes ownership of the `shoes` vector and `shoe_size` int
    // call `into_iter` creates an iterator that takes ownership of the vector.
    // calling collect gathers the values returned by the adapted 
    // iterator into a vector that’s returned by the function.
    shoes.into_iter().filter(|s| s.size == shoe_size).collect()
}

let shoes = vec![
    Shoe {
        size: 10,
        style: String::from("sneaker"),
    },
    Shoe {
        size: 13,
        style: String::from("sandal"),
    },
    Shoe {
        size: 10,
        style: String::from("boot"),
    },
];

assert_eq!(
    in_my_size,
    vec![
        Shoe {
            size: 10,
            style: String::from("sneaker")
        },
        Shoe {
            size: 10,
            style: String::from("boot")
        },
    ]
);

println!("in_my_size: {:?}", in_my_size);

in_my_size: [Shoe { size: 10, style: "sneaker" }, Shoe { size: 10, style: "boot" }]


## Comparing Performance: Loops vs Iterators

Iterators, although a high-level abstraction, get compiled down to roughly the same code as if you’d written the lower-level code yourself. Iterators are one of Rust’s zero-cost abstractions, by which we mean using the abstraction imposes no additional runtime overhead. The implementations of closures and iterators are such that runtime performance is not affected, and the compiled code for iterators are the same as the assemply code you would write by hand.